In [1]:
import pandas as pd
import FuzzyMatch as fm
import DatafileIO as dio
import ComposeFunction as cf
import ConfigReader as cr
from configparser import ConfigParser

In [2]:
companies = dio._load_files("tests/data/fuzzy_match_companies.xlsx")

companies.head(5)

,reference_name,new_name,expected
0,Ballentine Partners LLC,"Ballentine Partners, LLC",match
1,Huron Capital Partners LLC,Huron Capital,match
2,USI Insurance Svc,Usi Insurance Services,match
3,"New Heritage Capital, LLC",New Heritage Capital,match
4,Aureus,Aureus Capital,match


In [3]:
cp_fuzzy = cr.read_config("config/processing/fuzzy_match_company.ini")

fuzzy_fn = cr.parse_functions(cp_fuzzy)
fuzzy_clean = cr.parse_cleaning(cp_fuzzy)
fuzzy_clean_fn = cr.parse_functions(cr.read_config(fuzzy_clean))


print(fuzzy_fn)
print(fuzzy_clean)
print(fuzzy_clean_fn)

['FuzzyMatch.fuzzy_match_pairwise', 'FuzzyMatch.score_threshold']
['config/cleaning/clean_company_name.ini']
['StringFormat.to_lower', 'StringFormat.trim_whitespace', 'StringFormat.strip_non_ascii', 'StringFormat.strip_punctuation', 'FuzzyMatch.remove_company_suffixes']


In [4]:
path = "config/processing/fuzzy_match_company.ini"

cf.eval_functions_list(path, ['ConfigReader.read_config', 'ConfigReader.parse_cleaning', 'ConfigReader.read_config', 'ConfigReader.parse_functions'])

['StringFormat.to_lower',
 'StringFormat.trim_whitespace',
 'StringFormat.strip_non_ascii',
 'StringFormat.strip_punctuation',
 'FuzzyMatch.remove_company_suffixes']

In [5]:
cf.eval_functions_list(cp_fuzzy, ['ConfigReader.parse_cleaning', 'ConfigReader.read_config', 'ConfigReader.parse_functions'])

['StringFormat.to_lower',
 'StringFormat.trim_whitespace',
 'StringFormat.strip_non_ascii',
 'StringFormat.strip_punctuation',
 'FuzzyMatch.remove_company_suffixes']

In [6]:
path = "config/processing/fuzzy_match_company.ini"

dio._read_cleaning_from_ini(path)

['StringFormat.to_lower',
 'StringFormat.trim_whitespace',
 'StringFormat.strip_non_ascii',
 'StringFormat.strip_punctuation',
 'FuzzyMatch.remove_company_suffixes']

In [7]:
cp = ConfigParser()
emptycp = ConfigParser()
fm = cr.read_config("config/processing/fuzzy_match_company.ini")

In [8]:
list(cp.items())

[('DEFAULT', <Section: DEFAULT>)]

In [9]:
list(fm.items())

[('DEFAULT', <Section: DEFAULT>),
 ('info', <Section: info>),
 ('parameters', <Section: parameters>),
 ('control', <Section: control>)]